In [ ]:
import pandas as pd
import regex
import os
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')
import gensim
from gensim.models.phrases import Phrases, Phraser
from ast import literal_eval
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.tokenize import RegexpTokenizer
import re


tokenizer = RegexpTokenizer(r'\w+').tokenize
# tokenizer = word_tokenize

In [ ]:
path = "orignal dump"
df = pd.read_csv("Data.csv", index_col=0)

In [3]:
df.head()

,File,Season,SeasonID,Episode,EpisodeID,DialogueLines,Text,Word_count,Char_count,Stopwords
0,Suits.S01E01.srt,S01,1,E01,1,"[""Gerald Tate's here."", ""He wants to know what...",Gerald Tate's here. He wants to know what's ha...,9371,48607,3501
1,Suits.S01E02.srt,S01,1,E02,2,"['What is that, three in a row?', 'That would ...","What is that, three in a row? That would be fo...",6026,30980,2222
2,Suits.S01E03.srt,S01,1,E03,3,"[""♪ It's a perfect day ♪"", '♪ To go and tie on...",♪ It's a perfect day ♪ ♪ To go and tie one on ...,5843,30808,2150
3,Suits.S01E04.srt,S01,1,E04,4,"['Harvard trivia, the lightning round.', '[Sna...","Harvard trivia, the lightning round. [Snaps] G...",6469,34831,2307
4,Suits.S01E05.srt,S01,1,E05,5,"['Ross.', 'Triple double, courtesy of', ""Ameri...","Ross. Triple double, courtesy of America's fav...",5961,31022,2069


In [4]:
tokenized = []
for t in df["DialogueLines"]:
    ti = literal_eval(str(t).lower())
    tokenized.extend(ti)
tokenized = [tokenizer(t.replace("'","")) for t in tokenized]

In [5]:
tokenized[:10]

[['gerald', 'tates', 'here'],
 ['he', 'wants', 'to', 'know', 'whats', 'happening', 'to', 'his', 'deal'],
 ['go', 'get', 'harvey'],
 ['i', 'check'],
 ['raise'],
 ['im', 'all', 'in'],
 ['you', 'can', 'pay', 'me', 'later'],
 ['i', 'got', 'to', 'go'],
 ['gentlemen'],
 ['im', 'paying', 'you', 'millions', 'and', 'youre', 'telling', 'me']]

In [6]:
#to create the bigrams
bigram_model = Phrases(tokenized, delimiter=b" ")
bigram_sentences = []
bigram_sentences_t = []
#apply the trained model to a sentence
for t_sentence in tokenized:
    bigram_sentences.append(u' '.join(bigram_model[t_sentence]))
    bigram_sentences_t.append(bigram_model[t_sentence])
#get a trigram model out of the bigram
trigram_model = Phrases(bigram_sentences_t, delimiter=b" ")

trigram_sentences = []
trigram_sentences_t = []
#apply the trained model to a sentence
for t_sentence in bigram_sentences_t:
    trigram_sentences.append(u' '.join(trigram_model[t_sentence]))
    trigram_sentences_t.append(trigram_model[t_sentence])
#get a trigram model out of the bigram
fourgram_model = Phrases(trigram_sentences_t, delimiter=b" ")

In [7]:
def clean_text(text):
    text = text.lower()
    words = tokenizer(text)
    return " ".join(words)

df["Text_lower"] = df["Text"].apply(lambda x: clean_text(x))

In [8]:
# df["BiGramPhraser"] = df["Text"].apply(lambda x : bigram_model[tokenizer(x)]) 
# df["TriGramPhraser"] = df["Text"].apply(lambda x : trigram_model[tokenizer(x)]) 
# df["FourGramPhraser"] = df["Text"].apply(lambda x : fourgram_model[tokenizer(x)]) 

In [9]:
phrases = []
scores = []
for phrase, score in bigram_model.export_phrases(tokenized):
    phrases.append(phrase)
    scores.append(score)

for phrase, score in trigram_model.export_phrases(bigram_sentences_t):
    phrases.append(phrase)
    scores.append(score)

for phrase, score in fourgram_model.export_phrases(trigram_sentences_t):
    phrases.append(phrase)
    scores.append(score)

df_phrases = pd.DataFrame({"Phrase":phrases, "Score": scores, "Words": [len(p.split(b" ")) for p in phrases]})

In [10]:
df_phrases.sort_values(by=['Score'], inplace=True, ascending=False)

In [11]:
df_phrases = df_phrases.drop_duplicates()#head(20)
df_phrases = df_phrases.dropna()#head(20)
# df_phrases[df_phrases.isnull().any(axis=1)]
# df_phrases["Phrase"] = df_phrases["Phrase"]

In [12]:
junk_keyword_list = [b'web',
                    b'resync',
                    b'addic7ed',
                    b'font',
                    b'indistinct',
                    b'fffff',
                    b'tvsubtitles',
                    b'knock at door',
                    b'soft rock music',
                    b'air date',
                    b'color',
                    b'sync corrected',
                     b'00f'
                    ]
df_phrases.dtypes
# '|'.join(junk_keyword_list)

Phrase     object
Score     float64
Words       int64
dtype: object

In [13]:
df_phrases = df_phrases[~df_phrases['Phrase'].str.contains(b'|'.join(junk_keyword_list))]

In [14]:
df_phrases.head(10)

,Phrase,Score,Words
1936,b'wakefield cady',8318.172840,2
12781,b'cum laude',7895.765625,2
45508,b'shame shame shame shame',7290.897959,4
38939,b'blah blah blah',7217.252525,3
44626,b'quid pro quo',6379.535714,3
27937,b'van dyke',5614.766667,2
28352,b'hudson mills',5614.766667,2
13879,b'whistle blower',5614.766667,2
45226,b'having panic attacks',5582.093750,3
38640,b'elevator bell dings',5438.963141,3


In [15]:
TopScoreN = 100
for i in range(2, 6):
    Words = df_phrases.loc[df_phrases["Words"] == i , ["Phrase"]][:TopScoreN]
    Words = [w[0] for w in Words.values]
    print("Top Words with Length :%s\n"%(str(i)))
    print(Words)
    print("\n")

Top Words with Length :2

[b'wakefield cady', b'cum laude', b'van dyke', b'hudson mills', b'whistle blower', b'prune juice', b'silver platter', b'wrongful termination', b'footsteps approaching', b'reform corp', b'heartbeat thumping', b'stable shelters', b'starboard airlines', b'albert chung', b'clerical error', b'liberty rail', b'trent devon', b'panic attacks', b'rick sorkin', b'fletcher engines', b'stalking horse', b'effective immediately', b'olive branch', b'phillip allen', b'sloane moseley', b'holly cromwell', b'blah blah', b'sexual harassment', b'allison holt', b'vice president', b'sarah layton', b'lena lunders', b'waiter gillie', b'maria gomez', b'dana scott', b'clears throat', b'clifford danner', b'jesus christ', b'emerson petroleum', b'bike messenger', b'exhales sharply', b'paul porter', b'non compete', b'malicious prosecution', b'star trek', b'sam tull', b'sole purpose', b'shame shame', b'folsom foods', b'neil stillman', b'gavin andrews', b'monica eton', b'lola jensen', b'co co

In [16]:
def get_phrase_frequencies(phrase):
    try:
        phrase_regex = re.compile(phrase.decode("utf-8"))
        count = 0
        for episode in df["Text_lower"].values:
            c = 0
            c = len(re.findall(phrase_regex, episode))
            count = count + c
        return count
    except Exception as e:
#         print(phrase)
        raise

df_phrases["Freq"] = df_phrases["Phrase"].apply(lambda x: get_phrase_frequencies(x))

In [17]:
df_phrases.sort_values(by=['Freq', 'Score'], inplace=True, ascending=False)

In [18]:
TopScoreN = 100
for i in range(2, 6):
    Words = df_phrases.loc[df_phrases["Words"] == i , ["Phrase","Freq"]][:TopScoreN]
    Words = [(w[0], w[1]) for w in Words.values if len(w[0]) > 4]
    print("Top Words with Length :%s\n"%(str(i)))
    print(Words)
    print("\n")

Top Words with Length :2

[(b'talking about', 862), (b'right now', 773), (b'all right', 619), (b'of course', 345), (b'doing here', 342), (b'kind of', 338), (b'mike ross', 330), (b'talk about', 328), (b'your honor', 321), (b'first place', 314), (b'piece of', 285), (b'which means', 282), (b'excuse me', 261), (b'everybody wanna', 250), (b'look at', 248), (b'find out', 246), (b'care about', 242), (b'figure out', 233), (b'instead of', 231), (b'harvey specter', 230), (b'more than', 229), (b'my god', 229), (b'last night', 214), (b'son of', 201), (b'found out', 200), (b's office', 192), (b'he wants', 191), (b'your ass', 183), (b'managing partner', 181), (b'make sure', 178), (b'looking for', 176), (b'each other', 175), (b'my life', 172), (b'our own', 167), (b'how much', 167), (b'last time', 165), (b'senior partner', 161), (b'your wife', 160), (b'years ago', 159), (b'rest of', 158), (b'lot of', 158), (b'be able', 153), (b'same thing', 152), (b'take care', 149), (b'dramatic music', 148), (b'god d

In [19]:
df.head()

,File,Season,SeasonID,Episode,EpisodeID,DialogueLines,Text,Word_count,Char_count,Stopwords,Text_lower
0,Suits.S01E01.srt,S01,1,E01,1,"[""Gerald Tate's here."", ""He wants to know what...",Gerald Tate's here. He wants to know what's ha...,9371,48607,3501,gerald tate s here he wants to know what s hap...
1,Suits.S01E02.srt,S01,1,E02,2,"['What is that, three in a row?', 'That would ...","What is that, three in a row? That would be fo...",6026,30980,2222,what is that three in a row that would be four...
2,Suits.S01E03.srt,S01,1,E03,3,"[""♪ It's a perfect day ♪"", '♪ To go and tie on...",♪ It's a perfect day ♪ ♪ To go and tie one on ...,5843,30808,2150,it s a perfect day to go and tie one on the 20...
3,Suits.S01E04.srt,S01,1,E04,4,"['Harvard trivia, the lightning round.', '[Sna...","Harvard trivia, the lightning round. [Snaps] G...",6469,34831,2307,harvard trivia the lightning round snaps go pu...
4,Suits.S01E05.srt,S01,1,E05,5,"['Ross.', 'Triple double, courtesy of', ""Ameri...","Ross. Triple double, courtesy of America's fav...",5961,31022,2069,ross triple double courtesy of america s favor...
